In [7]:
import pandas as pd
import numpy as np

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

In [9]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Load your training dataset
train_df = pd.read_csv('new1_cyber_train.csv')
# Load your testing dataset
test_df = pd.read_csv('new1_cyber_test.csv')


In [11]:
train_df['label'] = train_df['label'].replace({0: 'bully', 1: 'not-bully'})
test_df['label'] = test_df['label'].replace({0: 'bully', 1: 'not-bully'})

In [12]:
# Data Preprocessing
# Find the maximum number of unique words in the dataset
max_words = 100

# Find the maximum length of messages in the dataset
max_len = 50


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


# Preprocessing for training data
train_df['text'] = train_df['text'].apply(preprocess_text)

# Split the training dataset
X_train, y_train = train_df['text'], train_df['label']

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Sequence and Padding for training data
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')

# Preprocessing for testing data
test_df['text'] = test_df['text'].apply(preprocess_text)

# Split the testing dataset
X_test, y_test = test_df['text'], test_df['label']

# Sequence and Padding for testing data
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')




## Building and Training Models

## DNN

In [6]:
# dnn_model_modified = Sequential([
#     Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
#     Flatten(),
#     Dense(32, activation='sigmoid'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dropout(0.7),  # Increased dropout rate
#     Dense(1, activation='sigmoid')
# ])

def create_dnn_model(optimizer='adam',learning_rate=0.001):
    dnn_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Flatten(),
        Dense(32, activation='sigmoid'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.7),  # Increased dropout rate
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    dnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

# Wrap the Keras model in a scikit-learn compatible classifier
dnn_classifier = KerasClassifier(build_fn=create_dnn_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_dnn = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_dnn = GridSearchCV(estimator=dnn_classifier, param_grid=param_grid_dnn, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_dnn = grid_search_dnn.fit(X_train_padded, y_train)


C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/30


19/19 - 1s - loss: 0.7778 - accuracy: 0.4942 - 1s/epoch - 61ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7257 - accuracy: 0.4925 - 27ms/epoch - 1ms/step
Epoch 3/30
19/19 - 0s - loss: 0.6951 - accuracy: 0.5341 - 28ms/epoch - 1ms/step
Epoch 4/30
19/19 - 0s - loss: 0.7020 - accuracy: 0.5008 - 25ms/epoch - 1ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6991 - accuracy: 0.4925 - 24ms/epoch - 1ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6922 - accuracy: 0.5275 - 25ms/epoch - 1ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6954 - accuracy: 0.4792 - 27ms/epoch - 1ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6933 - accuracy: 0.5324 - 33ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6936 - accuracy: 0.5008 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6971 - accuracy: 0.4975 - 33ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6923 - accuracy: 0.5291 - 30ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6973 - accuracy: 0.4942 - 30ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.8287 - accuracy: 0.4925 - 876ms/epoch - 46ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7307 - accuracy: 0.5058 - 35ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7082 - accuracy: 0.4725 - 31ms/epoch - 2ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6968 - accuracy: 0.4992 - 34ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5258 - 35ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6917 - accuracy: 0.5042 - 33ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6955 - accuracy: 0.4825 - 33ms/epoch - 2ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6902 - accuracy: 0.5225 - 33ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6905 - accuracy: 0.5308 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5258 - 33ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6939 - accuracy: 0.4975 - 29ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6896 - accuracy: 0.5125 - 31ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6895 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

19/19 - 1s - loss: 0.7451 - accuracy: 0.4809 - 784ms/epoch - 41ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7118 - accuracy: 0.5125 - 29ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7094 - accuracy: 0.5008 - 28ms/epoch - 1ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6926 - accuracy: 0.4958 - 29ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.7022 - accuracy: 0.5025 - 29ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6899 - accuracy: 0.5141 - 29ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6889 - accuracy: 0.5358 - 28ms/epoch - 1ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6940 - accuracy: 0.5291 - 31ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6877 - accuracy: 0.5308 - 30ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6853 - accuracy: 0.5607 - 30ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6869 - accuracy: 0.5408 - 30ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6905 - accuracy: 0.5308 - 29ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6874 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.7599 - accuracy: 0.5233 - 772ms/epoch - 41ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7272 - accuracy: 0.4850 - 29ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7123 - accuracy: 0.4767 - 30ms/epoch - 2ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6964 - accuracy: 0.5266 - 35ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6962 - accuracy: 0.5050 - 29ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6965 - accuracy: 0.5017 - 31ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6979 - accuracy: 0.4784 - 29ms/epoch - 2ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6909 - accuracy: 0.5083 - 30ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6940 - accuracy: 0.4618 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6932 - accuracy: 0.5183 - 29ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6898 - accuracy: 0.5199 - 29ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6984 - accuracy: 0.4884 - 30ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6925 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


29/29 - 1s - loss: 0.8094 - accuracy: 0.5044 - 769ms/epoch - 27ms/step
Epoch 2/30
29/29 - 0s - loss: 0.7247 - accuracy: 0.5089 - 41ms/epoch - 1ms/step
Epoch 3/30
29/29 - 0s - loss: 0.7084 - accuracy: 0.4701 - 43ms/epoch - 1ms/step
Epoch 4/30
29/29 - 0s - loss: 0.7008 - accuracy: 0.4834 - 42ms/epoch - 1ms/step
Epoch 5/30
29/29 - 0s - loss: 0.6963 - accuracy: 0.4800 - 42ms/epoch - 1ms/step
Epoch 6/30
29/29 - 0s - loss: 0.6969 - accuracy: 0.4867 - 41ms/epoch - 1ms/step
Epoch 7/30
29/29 - 0s - loss: 0.6964 - accuracy: 0.5000 - 43ms/epoch - 1ms/step
Epoch 8/30
29/29 - 0s - loss: 0.6951 - accuracy: 0.4945 - 50ms/epoch - 2ms/step
Epoch 9/30
29/29 - 0s - loss: 0.6911 - accuracy: 0.5200 - 52ms/epoch - 2ms/step
Epoch 10/30
29/29 - 0s - loss: 0.6910 - accuracy: 0.5100 - 51ms/epoch - 2ms/step
Epoch 11/30
29/29 - 0s - loss: 0.6930 - accuracy: 0.5000 - 45ms/epoch - 2ms/step
Epoch 12/30
29/29 - 0s - loss: 0.6865 - accuracy: 0.5266 - 43ms/epoch - 1ms/step
Epoch 13/30
29/29 - 0s - loss: 0.6737 - accura

In [7]:
# Display the results in a table
results_df_dnn = pd.DataFrame(grid_result_dnn.cv_results_)
results_df_dnn[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.743850,9
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.753865,5
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.764939,1
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.716216,14
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.666382,15
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.736109,10
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.763846,3
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.729439,12
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.752743,6
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.734969,11


In [8]:
# Print the best parameters and corresponding accuracy
print("Best Parameters (DNN): ", grid_result_dnn.best_params_)
print("Best Accuracy (DNN): ", grid_result_dnn.best_score_)

Best Parameters (DNN):  {'learning_rate': 0.01, 'optimizer': 'Adagrad'}
Best Accuracy (DNN):  0.7649390919158362


In [20]:
best_dnn_model = grid_result_dnn.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_dnn_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

8/8 - 0s - 49ms/epoch - 6ms/step
Test Accuracy: 0.8008849557522124
Precision: 0.75
Recall: 0.9026548672566371
F1-score: 0.8192771084337348


## LSTM

In [19]:
# lstm_model = Sequential([
#     Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
#     LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
#     Dropout(0.5),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])


def create_lstm_model(optimizer=Adam, learning_rate=0.001):
    lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),  # Increased output_dim for more expressive power
        LSTM(128, return_sequences=True),  # Increased LSTM units for more representational capacity
        Dropout(0.5),
        LSTM(128),  # Increased LSTM units for more representational capacity
        Dropout(0.5),  # Increased dropout rate for regularization
        Dense(64, activation='relu'),  # Increased number of neurons in Dense layers
        Dropout(0.3),  # Added dropout for regularization
        Dense(32, activation='relu'),  # Increased number of neurons in Dense layers
        Dropout(0.3),  # Added dropout for regularization
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_lstm = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_lstm = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid_lstm, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_lstm = grid_search_lstm.fit(X_train_padded, y_train)


C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 4s - loss: 0.6969 - accuracy: 0.4925 - 4s/epoch - 206ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.5058 - 1s/epoch - 55ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6947 - accuracy: 0.5125 - 1s/epoch - 55ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6952 - accuracy: 0.4892 - 972ms/epoch - 51ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6936 - accuracy: 0.5125 - 1s/epoch - 53ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6936 - accuracy: 0.4925 - 1s/epoch - 53ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6922 - accuracy: 0.5008 - 1s/epoch - 54ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6936 - accuracy: 0.5008 - 1s/epoch - 54ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6835 - accuracy: 0.5474 - 1s/epoch - 58ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6757 - accuracy: 0.5424 - 1s/epoch - 59ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6493 - accuracy: 0.5707 - 1s/epoch - 58ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6765 - accuracy: 0.5358 - 1s/epoch - 59ms/step
Epoch 13/30
19/19 - 1s - loss: 0.7448 - accur

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 5s - loss: 0.6945 - accuracy: 0.5075 - 5s/epoch - 260ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6947 - accuracy: 0.5075 - 1s/epoch - 56ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6940 - accuracy: 0.4859 - 1s/epoch - 57ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6945 - accuracy: 0.4975 - 1s/epoch - 57ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6962 - accuracy: 0.4992 - 1s/epoch - 56ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6943 - accuracy: 0.4775 - 1s/epoch - 56ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6920 - accuracy: 0.5341 - 1s/epoch - 56ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6948 - accuracy: 0.5075 - 1s/epoch - 56ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6939 - accuracy: 0.4908 - 1s/epoch - 56ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.5241 - 1s/epoch - 56ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6892 - accuracy: 0.4809 - 1s/epoch - 56ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.5158 - 1s/epoch - 56ms/step
Epoch 13/30
19/19 - 1s - loss: 0.6914 - accuracy

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
19/19 - 4s - loss: 0.6948 - accuracy: 0.4742 - 4s/epoch - 225ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6948 - accuracy: 0.4942 - 862ms/epoch - 45ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6944 - accuracy: 0.4908 - 864ms/epoch - 45ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6941 - accuracy: 0.5008 - 867ms/epoch - 46ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6933 - accuracy: 0.5291 - 855ms/epoch - 45ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6945 - accuracy: 0.4825 - 866ms/epoch - 46ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6936 - accuracy: 0.4809 - 869ms/epoch - 46ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6938 - accuracy: 0.5008 - 855ms/epoch - 45ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.5108 - 866ms/epoch - 46ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6939 - accuracy: 0.5075 - 859ms/epoch - 45ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6929 - accuracy: 0.5308 - 857ms/epoch - 45ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6927 - accuracy: 0.5208 - 864ms/epoch - 45ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 4s - loss: 0.6951 - accuracy: 0.4867 - 4s/epoch - 201ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6960 - accuracy: 0.4801 - 875ms/epoch - 46ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6942 - accuracy: 0.4784 - 856ms/epoch - 45ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6958 - accuracy: 0.5000 - 856ms/epoch - 45ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6947 - accuracy: 0.4950 - 869ms/epoch - 46ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6953 - accuracy: 0.4784 - 869ms/epoch - 46ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6944 - accuracy: 0.4867 - 867ms/epoch - 46ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6944 - accuracy: 0.4834 - 864ms/epoch - 45ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6950 - accuracy: 0.4834 - 856ms/epoch - 45ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6919 - accuracy: 0.5150 - 852ms/epoch - 45ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6756 - accuracy: 0.6080 - 847ms/epoch - 45ms/step
Epoch 12/30
19/19 - 1s - loss: 0.5875 - accuracy: 0.6777 - 858ms/epoch - 45ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
29/29 - 4s - loss: 0.6947 - accuracy: 0.4568 - 4s/epoch - 145ms/step
Epoch 2/30
29/29 - 1s - loss: 0.6942 - accuracy: 0.5078 - 1s/epoch - 43ms/step
Epoch 3/30
29/29 - 1s - loss: 0.6948 - accuracy: 0.5078 - 1s/epoch - 44ms/step
Epoch 4/30
29/29 - 1s - loss: 0.6938 - accuracy: 0.5111 - 1s/epoch - 44ms/step
Epoch 5/30
29/29 - 1s - loss: 0.6940 - accuracy: 0.4911 - 1s/epoch - 43ms/step
Epoch 6/30
29/29 - 1s - loss: 0.6936 - accuracy: 0.4867 - 1s/epoch - 44ms/step
Epoch 7/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5133 - 1s/epoch - 44ms/step
Epoch 8/30
29/29 - 1s - loss: 0.6938 - accuracy: 0.5044 - 1s/epoch - 45ms/step
Epoch 9/30
29/29 - 1s - loss: 0.6938 - accuracy: 0.4745 - 1s/epoch - 43ms/step
Epoch 10/30
29/29 - 1s - loss: 0.6923 - accuracy: 0.5044 - 1s/epoch - 44ms/step
Epoch 11/30
29/29 - 1s - loss: 0.6895 - accuracy: 0.5133 - 1s/epoch - 43ms/step
Epoch 12/30
29/29 - 1s - loss: 0.6890 - accuracy: 0.4878 - 1s/epoch - 44ms/step
Epoch 13/30
29/29 - 1s - loss: 0.6840 - accuracy

In [20]:
results_df_lstm = pd.DataFrame(grid_result_lstm.cv_results_)
results_df_lstm[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.587770,11
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.531015,13
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.599985,7
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.681883,3
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.658383,5
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.511096,14
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.675242,4
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.616571,6
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.580842,12
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.596633,9


In [21]:
print("Best Parameters (LSTM): ", grid_result_lstm.best_params_)
print("Best Accuracy (LSTM): ", grid_result_lstm.best_score_)

Best Parameters (LSTM):  {'learning_rate': 0.0001, 'optimizer': 'RMSprop'}
Best Accuracy (LSTM):  0.7572093023255814


In [22]:
best_lstm_model = grid_result_lstm.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_lstm_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)


8/8 - 1s - 714ms/epoch - 89ms/step
Test Accuracy: 0.7079646017699115


In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='bully')
recall_spam = recall_score(y_test, y_pred, pos_label='bully')
f1_spam = f1_score(y_test, y_pred, pos_label='bully')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not-bully')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not-bully')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not-bully')

print("Precision (bully):", precision_spam)
print("Recall (bully):", recall_spam)
print("F1-score (bully):", f1_spam)

print("Precision (not-bully):", precision_non_spam)
print("Recall (not-bully):", recall_non_spam)
print("F1-score (not-bully):", f1_non_spam)

Precision (bully): 0.9433962264150944
Recall (bully): 0.4424778761061947
F1-score (bully): 0.6024096385542169
Precision (not-bully): 0.6358381502890174
Recall (not-bully): 0.9734513274336283
F1-score (not-bully): 0.7692307692307693


## Bi-LSTM

In [13]:
def create_bi_lstm_model(optimizer=Adam, learning_rate=0.001, lstm_units=64, dropout_rate=0.3):
    bi_lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Bidirectional(LSTM(units=lstm_units, return_sequences=True)),
        Bidirectional(LSTM(units=lstm_units)),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    
    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'
    
    bi_lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return bi_lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
bi_lstm_classifier = KerasClassifier(build_fn=create_bi_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search = GridSearchCV(estimator=bi_lstm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result = grid_search.fit(X_train_padded, y_train)



C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/30


19/19 - 6s - loss: 0.6932 - accuracy: 0.5075 - 6s/epoch - 316ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6902 - accuracy: 0.5557 - 515ms/epoch - 27ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6545 - accuracy: 0.6656 - 507ms/epoch - 27ms/step
Epoch 4/30
19/19 - 1s - loss: 0.5171 - accuracy: 0.7421 - 506ms/epoch - 27ms/step
Epoch 5/30
19/19 - 1s - loss: 0.4732 - accuracy: 0.7953 - 507ms/epoch - 27ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4573 - accuracy: 0.8220 - 521ms/epoch - 27ms/step
Epoch 7/30
19/19 - 1s - loss: 0.4601 - accuracy: 0.8236 - 525ms/epoch - 28ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4334 - accuracy: 0.8203 - 514ms/epoch - 27ms/step
Epoch 9/30
19/19 - 1s - loss: 0.5686 - accuracy: 0.7404 - 504ms/epoch - 27ms/step
Epoch 10/30
19/19 - 1s - loss: 0.5771 - accuracy: 0.6922 - 514ms/epoch - 27ms/step
Epoch 11/30
19/19 - 1s - loss: 0.4982 - accuracy: 0.7720 - 501ms/epoch - 26ms/step
Epoch 12/30
19/19 - 1s - loss: 0.4411 - accuracy: 0.8136 - 525ms/epoch - 28ms/step
Epoch 13/30

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 6s - loss: 0.6931 - accuracy: 0.4775 - 6s/epoch - 342ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6910 - accuracy: 0.5341 - 577ms/epoch - 30ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6795 - accuracy: 0.6140 - 573ms/epoch - 30ms/step
Epoch 4/30
19/19 - 1s - loss: 0.5603 - accuracy: 0.7504 - 574ms/epoch - 30ms/step
Epoch 5/30
19/19 - 1s - loss: 0.4654 - accuracy: 0.7937 - 562ms/epoch - 30ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4571 - accuracy: 0.7887 - 556ms/epoch - 29ms/step
Epoch 7/30
19/19 - 1s - loss: 0.4324 - accuracy: 0.8020 - 553ms/epoch - 29ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4144 - accuracy: 0.8103 - 554ms/epoch - 29ms/step
Epoch 9/30
19/19 - 1s - loss: 0.4096 - accuracy: 0.8286 - 557ms/epoch - 29ms/step
Epoch 10/30
19/19 - 1s - loss: 0.4118 - accuracy: 0.8270 - 568ms/epoch - 30ms/step
Epoch 11/30
19/19 - 1s - loss: 0.4087 - accuracy: 0.8203 - 572ms/epoch - 30ms/step
Epoch 12/30
19/19 - 1s - loss: 0.3998 - accuracy: 0.8270 - 580ms/epoch - 31ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
19/19 - 7s - loss: 0.6945 - accuracy: 0.4992 - 7s/epoch - 365ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6905 - accuracy: 0.5391 - 826ms/epoch - 43ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6876 - accuracy: 0.5458 - 833ms/epoch - 44ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6420 - accuracy: 0.6339 - 831ms/epoch - 44ms/step
Epoch 5/30
19/19 - 1s - loss: 0.4711 - accuracy: 0.7720 - 835ms/epoch - 44ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4457 - accuracy: 0.8136 - 834ms/epoch - 44ms/step
Epoch 7/30
19/19 - 1s - loss: 0.4297 - accuracy: 0.8003 - 830ms/epoch - 44ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4238 - accuracy: 0.8253 - 842ms/epoch - 44ms/step
Epoch 9/30
19/19 - 1s - loss: 0.4198 - accuracy: 0.8270 - 833ms/epoch - 44ms/step
Epoch 10/30
19/19 - 1s - loss: 0.3980 - accuracy: 0.8369 - 826ms/epoch - 43ms/step
Epoch 11/30
19/19 - 1s - loss: 0.3852 - accuracy: 0.8436 - 831ms/epoch - 44ms/step
Epoch 12/30
19/19 - 1s - loss: 0.3976 - accuracy: 0.8369 - 822ms/epoch - 43ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 8s - loss: 0.6938 - accuracy: 0.4950 - 8s/epoch - 403ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6910 - accuracy: 0.5365 - 852ms/epoch - 45ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6679 - accuracy: 0.6711 - 858ms/epoch - 45ms/step
Epoch 4/30
19/19 - 1s - loss: 0.5364 - accuracy: 0.7176 - 864ms/epoch - 45ms/step
Epoch 5/30
19/19 - 1s - loss: 0.5256 - accuracy: 0.7176 - 863ms/epoch - 45ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4818 - accuracy: 0.8073 - 862ms/epoch - 45ms/step
Epoch 7/30
19/19 - 1s - loss: 0.4679 - accuracy: 0.7857 - 858ms/epoch - 45ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4311 - accuracy: 0.8272 - 851ms/epoch - 45ms/step
Epoch 9/30
19/19 - 1s - loss: 0.4184 - accuracy: 0.8405 - 858ms/epoch - 45ms/step
Epoch 10/30
19/19 - 1s - loss: 0.4066 - accuracy: 0.8439 - 854ms/epoch - 45ms/step
Epoch 11/30
19/19 - 1s - loss: 0.3946 - accuracy: 0.8488 - 855ms/epoch - 45ms/step
Epoch 12/30
19/19 - 1s - loss: 0.4048 - accuracy: 0.8289 - 860ms/epoch - 45ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
29/29 - 7s - loss: 0.6947 - accuracy: 0.4911 - 7s/epoch - 257ms/step
Epoch 2/30
29/29 - 1s - loss: 0.6915 - accuracy: 0.5521 - 1s/epoch - 46ms/step
Epoch 3/30
29/29 - 1s - loss: 0.6235 - accuracy: 0.6962 - 1s/epoch - 46ms/step
Epoch 4/30
29/29 - 1s - loss: 0.4745 - accuracy: 0.7871 - 1s/epoch - 45ms/step
Epoch 5/30
29/29 - 1s - loss: 0.4502 - accuracy: 0.7971 - 1s/epoch - 46ms/step
Epoch 6/30
29/29 - 1s - loss: 0.4299 - accuracy: 0.8004 - 1s/epoch - 45ms/step
Epoch 7/30
29/29 - 1s - loss: 0.4277 - accuracy: 0.8071 - 1s/epoch - 46ms/step
Epoch 8/30
29/29 - 1s - loss: 0.4207 - accuracy: 0.8115 - 1s/epoch - 45ms/step
Epoch 9/30
29/29 - 1s - loss: 0.4227 - accuracy: 0.8104 - 1s/epoch - 46ms/step
Epoch 10/30
29/29 - 1s - loss: 0.4292 - accuracy: 0.8082 - 1s/epoch - 45ms/step
Epoch 11/30
29/29 - 1s - loss: 0.4126 - accuracy: 0.8149 - 1s/epoch - 46ms/step
Epoch 12/30
29/29 - 1s - loss: 0.4183 - accuracy: 0.8149 - 1s/epoch - 45ms/step
Epoch 13/30
29/29 - 1s - loss: 0.4208 - accuracy

In [14]:
results_df = pd.DataFrame(grid_result.cv_results_)
results_df[['params', 'mean_test_score', 'rank_test_score']]
# Print the best parameters and corresponding accuracy

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.780476,2
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.763872,14
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.772735,10
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.776061,6
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.776069,5
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.776054,7
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.773843,9
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.774932,8
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.779387,3
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.768295,12


In [15]:
print("\n",grid_result)
print("\n")
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)



 GridSearchCV(cv=3,
             estimator=KerasClassifier(build_fn=<function create_bi_lstm_model at 0x000001E70BDC3370>, epochs=30, learning_rate=0.01, verbose=2),
             param_grid={'learning_rate': [0.01, 0.001, 0.0001],
                         'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta',
                                       'Nadam']},
             scoring='accuracy')


Best Parameters:  {'learning_rate': 0.0001, 'optimizer': 'RMSprop'}
Best Accuracy:  0.7815946843853819


In [17]:
best_model = grid_result.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)

from sklearn.metrics import precision_score, recall_score, f1_score


8/8 - 0s - 142ms/epoch - 18ms/step
Test Accuracy: 0.7920353982300885


In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='bully')
recall_spam = recall_score(y_test, y_pred, pos_label='bully')
f1_spam = f1_score(y_test, y_pred, pos_label='bully')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not-bully')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not-bully')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not-bully')

print("Precision (bully):", precision_spam)
print("Recall (bully):", recall_spam)
print("F1-score (bully):", f1_spam)

print("Precision (not-bully):", precision_non_spam)
print("Recall (not-bully):", recall_non_spam)
print("F1-score (not-bully):", f1_non_spam)

Precision (bully): 0.8666666666666667
Recall (bully): 0.6902654867256637
F1-score (bully): 0.768472906403941
Precision (not-bully): 0.7426470588235294
Recall (not-bully): 0.8938053097345132
F1-score (not-bully): 0.8112449799196788
